# 1. Cuestionario del alumno

In [1]:
from scipy import spatial
import numpy as np

In [2]:
BD = "proyecto_final"

In [3]:
dic_campos_bd = {
     1: "arte_diseno_arquitectura",
     2: "agricultura_forestal",
     3: "empresariales_administracion",
     4: "economicas",
     5: "educacion",
     6: "ingenieria_tecnologia",
     7: "historia_filosofia_teologia",
     8: "leyes_abogacia",
     9: "ciencias_salud",
    10: "ciencias_naturales",
    11: "ciencias_sociales",
    12: "otros",
}

dic_campos_usuario = {
    "arte_diseno_arquitectura": "Arte, diseño y arquitectura",
    "agricultura_forestal": "Agricultura y forestal",
    "empresariales_administracion": "Empresariales y administración",
    "economicas": "Economía",
    "educacion": "Educación",
    "ingenieria_tecnologia": "Ingeniería y tecnología",
    "historia_filosofia_teologia": "Historia, filosofía y teología",
    "leyes_abogacia": "Leyes y abogacía",
    "ciencias_salud": "Ciencias de la salud",
    "ciencias_naturales": "Ciencias naturales",
    "ciencias_sociales": "Ciencias sociales",
    "otros": "Otros",
}

In [4]:
print("Introduce el identificador de 3 campos de tu interés de mayor a menor preferencia:")
for key, value in dic_campos_bd.items():
    print(f"{'{:3d}'.format(key)}. {dic_campos_usuario[value]}")
print()

completed = False
selected_ids = []
max_length = 3
while not completed:
    selection = input(f"Introduce el identificador de tu elección ({len(selected_ids)}/{max_length}): ")
    if selection not in selected_ids:
        print(f"    Has elegido {dic_campos_usuario[dic_campos_bd[int(selection)]]}")
        selected_ids.append(selection)
        if len(selected_ids) == max_length:
            completed = True
    else:
        print("    Error: Ya has escogido ese campo")

Introduce el identificador de 3 campos de tu interés de mayor a menor preferencia:
  1. Arte, diseño y arquitectura
  2. Agricultura y forestal
  3. Empresariales y administración
  4. Economía
  5. Educación
  6. Ingeniería y tecnología
  7. Historia, filosofía y teología
  8. Leyes y abogacía
  9. Ciencias de la salud
 10. Ciencias naturales
 11. Ciencias sociales
 12. Otros

Introduce el identificador de tu elección (0/3): 4
    Has elegido Economía
Introduce el identificador de tu elección (1/3): 2
    Has elegido Agricultura y forestal
Introduce el identificador de tu elección (2/3): 6
    Has elegido Ingeniería y tecnología


In [26]:
proj_usuario = [0.5, 0.25, 0.1]

Sentencia para obtener el ranking de las universidades en los campos elegidos por el usuario

In [6]:
sentencia_select = f"SELECT {BD}.universidades.nombre AS nombre_univ, {BD}.campos.nombre AS nombre_campo, {BD}.rankings.id_univ, {BD}.rankings.id_campo, {BD}.rankings.puntuacion FROM {BD}.campos"
sentencia_join = f"""INNER JOIN {BD}.rankings ON {BD}.campos.id = {BD}.rankings.id_campo
                     INNER JOIN {BD}.universidades ON {BD}.universidades.id = {BD}.rankings.id_univ"""
ids_str = ','.join(selected_ids)
sentencia_where = f"WHERE {BD}.campos.id IN ({','.join(selected_ids)})"

sentencia_sql = f"""{sentencia_select}
                    {sentencia_join}
                    {sentencia_where}"""


rankings_df = spark.sql(sentencia_sql).fillna(0)
rankings_df.show()

2023-07-25 23:53:12,232 WARN conf.HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
2023-07-25 23:53:12,233 WARN conf.HiveConf: HiveConf of name hive.stats.retries.wait does not exist
2023-07-25 23:53:14,031 WARN metastore.ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


+--------------------+--------------------+-------+--------+----------+
|         nombre_univ|        nombre_campo|id_univ|id_campo|puntuacion|
+--------------------+--------------------+-------+--------+----------+
|Universidad Nacio...|agricultura_forestal|     36|       2|         0|
|Universidad Nacio...|          economicas|     36|       4|         5|
|Universidad Nacio...|ingenieria_tecnol...|     36|       6|        23|
|Universidad Obert...|agricultura_forestal|     37|       2|         0|
|Universidad Obert...|          economicas|     37|       4|         4|
|Universidad Obert...|ingenieria_tecnol...|     37|       6|        17|
|Universidad Polit...|agricultura_forestal|     41|       2|        26|
|Universidad Polit...|          economicas|     41|       4|         1|
|Universidad Polit...|ingenieria_tecnol...|     41|       6|        58|
|Universidad de Ba...|agricultura_forestal|     57|       2|        15|
|Universidad de Ba...|          economicas|     57|       4|    

In [7]:
list_of_unis = zip(*(iter(rankings_df.collect()),) * 3)

In [8]:
unis_dict = {}
for campo_1, campo_2, campo_3 in list_of_unis:
    uni_dict = {campo_1['id_campo']: campo_1['puntuacion'],
                campo_2['id_campo']: campo_2['puntuacion'],
                campo_3['id_campo']: campo_3['puntuacion'],
                }
    unis_dict[campo_1['id_univ']] = uni_dict

In [9]:
def calculate_uni_projection(uni_dict, selected_ids):
    """
        Creates a projection of the university calculating the porcentage of investigators in each field selected
        by the user
    """
    total_investigators = 0
    for investigators in uni_dict.values():
        total_investigators += investigators
    
    if total_investigators == 0:
        return [0, 0, 0]
    # For each field, ordered by user preference, calculate the percentage of total investigators
    uni_proj = []
    for campo in selected_ids:
        percentage = uni_dict[int(campo)]/total_investigators
        uni_proj.append(percentage)
        
    return uni_proj

In [23]:
projs_dict = {}
for key, value in unis_dict.items():
    projs_dict[key] = calculate_uni_projection(value, selected_ids)
print(projs_dict)

{36: [0.17857142857142858, 0.0, 0.8214285714285714], 37: [0.19047619047619047, 0.0, 0.8095238095238095], 41: [0.011764705882352941, 0.3058823529411765, 0.6823529411764706], 57: [0.2072072072072072, 0.13513513513513514, 0.6576576576576577], 66: [0.2962962962962963, 0.0, 0.7037037037037037], 74: [0.1111111111111111, 0.1111111111111111, 0.7777777777777778], 77: [0.20238095238095238, 0.10714285714285714, 0.6904761904761905], 87: [1.0, 0.0, 0.0], 8: [1.0, 0.0, 0.0], 10: [0.16666666666666666, 0.25, 0.5833333333333334], 11: [0.21323529411764705, 0.07352941176470588, 0.7132352941176471], 12: [0, 0, 0], 13: [0.0, 0.0, 1.0], 14: [0.24519230769230768, 0.0, 0.7548076923076923], 16: [0, 0, 0], 51: [0.5, 0.0, 0.5], 44: [0.0, 0.0, 1.0], 18: [0.2150170648464164, 0.040955631399317405, 0.7440273037542662], 2: [0.0, 0.0, 1.0], 3: [0, 0, 0], 22: [0.0, 0.0, 1.0], 24: [0.5, 0.0, 0.5], 21: [0, 0, 0], 23: [0, 0, 0], 26: [1.0, 0.0, 0.0], 5: [0, 0, 0], 30: [0.25, 0.0, 0.75], 32: [0.26666666666666666, 0.0, 0.733

In [44]:
def get_three_most_similar_unis(projs_dict, proj_user):
    """
        Searches for the three most similar unis to the user preferences
    """
    similarities_dict = {}
    for uni_key, proj in projs_dict.items():
        similarities_dict[uni_key] = spatial.distance.cosine(proj_user, proj)
        
    similarities_ordered = sorted(similarities_dict.items(), key=lambda x:x[1], reverse=True)
    return similarities_ordered

In [45]:
sims = get_three_most_similar_unis(projs_dict, proj_usuario)
print(sims)

[(13, 0.8239098187348752), (44, 0.8239098187348752), (2, 0.8239098187348752), (22, 0.8239098187348752), (52, 0.8239098187348752), (45, 0.8239098187348752), (50, 0.8239098187348752), (28, 0.8239098187348752), (7, 0.8239098187348752), (40, 0.8147031352578279), (15, 0.7809796510129783), (47, 0.725273972711157), (61, 0.7139741211710381), (49, 0.7115121020157629), (42, 0.6777721300342667), (65, 0.6655766836731869), (41, 0.6454310633363312), (74, 0.6424654179632673), (36, 0.6408949640671411), (58, 0.6408949640671411), (67, 0.6386771522285891), (59, 0.6292088148485004), (82, 0.6272554446169081), (37, 0.6269329054186811), (81, 0.618422199550664), (70, 0.6105684849703191), (53, 0.6040442506427812), (79, 0.5961015498882904), (84, 0.588806607725125), (80, 0.5882975486563098), (68, 0.5683153836626595), (18, 0.5637222808422864), (14, 0.5605098932358866), (30, 0.5545231628882364), (11, 0.5378571620198725), (32, 0.5336236160809653), (86, 0.5296535085384835), (77, 0.5230795096355645), (72, 0.517101915

/home/student/.local/lib/python3.8/site-packages/scipy/spatial/distance.py:622: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [43]:
print(sentencia_sql)

SELECT proyecto_final.universidades.nombre AS nombre_univ, proyecto_final.campos.nombre AS nombre_campo, proyecto_final.rankings.id_univ, proyecto_final.rankings.id_campo, proyecto_final.rankings.puntuacion FROM proyecto_final.campos
                    INNER JOIN proyecto_final.rankings ON proyecto_final.campos.id = proyecto_final.rankings.id_campo
                     INNER JOIN proyecto_final.universidades ON proyecto_final.universidades.id = proyecto_final.rankings.id_univ
                    WHERE proyecto_final.campos.id IN (4,2,6)
